In [42]:
import yaml
from sklearn.model_selection import ParameterGrid, ParameterSampler
from config import Config
import uuid
from subprocess import check_call
import shutil

In [46]:
config = Config("config.yaml")

In [50]:
config = Config("sparsification.yaml")

In [52]:
meta, hp = config.meta, config.hp
for i in range(meta.repeat):
    if meta.mode == "grid":
        config_list = list(ParameterGrid(hp))
    elif meta.mode == "random":
        config_list = list(ParameterSampler(hp, meta.sample))
        
    for idx, c in enumerate(config_list):
        if meta.key is not None:
            key_config = meta.key
            c['exp_name'] = exp_name = "_".join([str(c[k]) for k in key_config]) + f"_round_{i}"
        else:
            c['exp_name'] = exp_name = uuid.uuid4().hex
        exec_line = f"python {meta.script} {meta.func} " + " ".join([f"--{k} {v}" for k, v in c.items()])
        
        """
        with open(f'{meta.sandbox}/temp.sh', 'w') as outfile:
            outfile.write("#!/bin/bash\n")
            outfile.write(f"rm -rf {meta.sandbox}/{exp_name}\n")
            outfile.write(f"rm -rf {meta.sandbox}/tb/{exp_name}\n")
            outfile.write(exec_line)
        """
        
        with open("sbatch_template.sh") as f:
            template = f.read()

            template = template.replace("@GPUS", f"{meta.gpus}")
            template = template.replace("@LOG", f"{meta.logpath}")
            template = template.replace("@NAME", f"{exp_name}")
            template = template.replace("@COMMAND", f"{exec_line}")
            
            sbatch_filepath = os.path.join(meta.sandbox, f"{exp_name}.sh")
            with open(sbatch_filepath, 'w') as f:
                f.write(template)
                
            check_call(f"sbatch {sbatch_filepath}", shell=True)
            shutil.rmtree(sbatch_filepath)

In [28]:


check_call("srun /usr/local/cuda/samples/1_Utilities/deviceQuery/deviceQuery", shell=True)

CalledProcessError: Command 'srun /usr/local/cuda/samples/1_Utilities/deviceQuery/deviceQuery' returned non-zero exit status 1.

In [29]:
os.system("srun /usr/local/cuda/samples/1_Utilities/deviceQuery/deviceQuery")

256

In [ ]:
for i in range(1,2):
    for idx, c in tqdm(enumerate(list(ParameterGrid(config.hp)))):
        key_config = ["model", "backbone", "lr", "n_groups"]
        exp_name = "_".join(["VI"] + [str(c[k]) for k in key_config]) + f"_exp{i}"
        c['exp_name'] = exp_name
        exec_line = f"python {meta.script} train" + " ".join([f"--{k} {v}" for k, v in c.items()])
        
        with open(f'{meta.sandbox}/temp.sh', 'w') as outfile:
            outfile.write("#!/bin/bash\n")
            outfile.write(f"rm -rf {meta.sandbox}/{exp_name}\n")
            outfile.write(f"rm -rf {meta.sandbox}/tb/{exp_name}\n")
            outfile.write(exec_line)
        
        os.system('sbatch /sailhome/haosheng/workspace/agriculture/scripts/run_temp_vi.sh')
        time.sleep(3)

In [30]:
subprocess.call(["ls", "-l"])

NameError: name 'subprocess' is not defined